#### Instructions :
* Please ruin this notebook in google colab as the notebook code is implemented in that fashion
* for running the notebook , we need some data files that needs to be stored in the google drive location and needs to be mounted before running the notebook.
1. NLP dataq zip file contents : [slang, Emoji_Dict.p]
2. twitter

In [1]:
!pip install GetOldTweets3 snscrape wikipedia ekphrasis emot emoji pmdarima statsmodels torch attrdict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.4 MB/s 
     |████████████████████████████████| 61 kB 15 kB/s 
     |████████████████████████████████| 175 kB 57.3 MB/s 
     |████████████████████████████████| 1.4 MB 35.9 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 9.8 MB 49.1 MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=7a545c07d3a436a6425ab33ef56396922232c24b82eb6513779eadcb0cb9d1a2
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=87af5531fa02c119b0428cc3d9debd6c57c078f916b529ec4b3414ef9e5ac2b6
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e21

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

DRIVE_PATH = '/content/gdrive/MyDrive/Semester 3/Financial Econometrics/HW4 - Wave exam'

#import sys
#sys.path.insert(0,'/content/gdrive/MyDrive/Semester 3/Financial Econometrics/HW4 - Wave exam')
#%cd /content/gdrive/MyDrive/Semester\ 3/Financial\ Econometrics/HW4\ -\ Wave\ exam
#%ls

Mounted at /content/gdrive/


In [3]:
## GoEmotions Setup ##
!git clone https://github.com/monologg/GoEmotions-pytorch.git
# go to the cloned Repo of GoEmotions
%cd /content/GoEmotions-pytorch
!pip install torch #--force--reinstall
!pip install transformers #--force--reinstall
!pip install attrdict

Cloning into 'GoEmotions-pytorch'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.
/content/GoEmotions-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.7 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import GetOldTweets3 as got
import pandas as pd
import csv
from datetime import date, timedelta, datetime
import re 
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import tweepy
import snscrape.modules.twitter as sntwitter
import wikipedia
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
tqdm.pandas()
import os.path
from os import path
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import emoji
import re
import en_core_web_sm
import string
import string as letter
import pickle
from ekphrasis.classes.segmenter import Segmenter
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from functools import partial

import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf, adfuller, grangercausalitytests
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from sklearn.preprocessing import StandardScaler

# For the GoEmojtion (path is set inside the git repo)
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline

import nltk
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

# Extracting Twitter Data

## Twitters Personal API

In [5]:
# Twitter API
config_twitter = {
    "_comment": "create you account in https://developer.twitter.com/",
    "api_key": "TxjfOc7q4Ng6Tn2sqjDrXhgON",
    "api_secret": "1aITGcE5sfNwrGBtO9AIevtQX8kxeUsvFifroQ3nQrsAKqMMBV",
    "access_token": "1401134287718658048-gmHn7QeXOyUhwt5NEkLQIQIGgPRmyB",
    "access_secret": "5rErqN1iBzb8AMOGo986MxKrAYz13iaJjKOLJw5EPUz23"
}

In [6]:
def get_credentials():
    auth = tweepy.OAuthHandler(config_twitter['api_key'], config_twitter['api_secret'])
    auth.set_access_token(config_twitter['access_token'], config_twitter['access_secret'])
    return tweepy.API(auth, wait_on_rate_limit = True)

## Scrape using snTwitter

#### Parse out Senates from .csv files and find unique Twitter accounts

In [ ]:
# 2 month old Kaggle
# https://www.kaggle.com/competitions/congressionaltweetcompetitionspring2022/data
#congress_kaggle = pd.read_csv(f"/content/gdrive/MyDrive/Semester 3/Financial Econometrics/HW4 - Wave exam/data/senates/congressional_tweet_training_data.csv")

#congress_kaggle = pd.read_csv(f"{DRIVE_PATH}FTSE100-Daily.csv")
#print(congress_kaggle.head(5))

# https://github.com/pablobarbera/twitter_ideology/blob/master/primary/data/congress-social-media.csv
congress_csv = pd.read_csv(f"/content/gdrive/MyDrive/Semester 3/Financial Econometrics/HW4 - Wave exam/data/senates/congress-social-media.csv")
print(congress_csv.head(5))

    id    bioid                   name gender type       party  \
0   62  B000213             Joe Barton      M  rep  Republican   
1   70  B000287         Xavier Becerra      M  rep    Democrat   
2   91  B000490  Sanford D. Bishop Jr.      M  rep    Democrat   
3   99  B000574        Earl Blumenauer      M  rep    Democrat   
4  102  B000589        John A. Boehner      M  rep  Republican   

           twitter  
0     RepJoeBarton  
1       RepBecerra  
2    SanfordBishop  
3  BlumenauerMedia  
4   SpeakerBoehner  


### Get Accounts to Filter by

In [ ]:
# Find the Twitters Most Popular Accounts #
# Currently from Wikipedia
# https://en.wikipedia.org/wiki/List_of_most-followed_Twitter_accounts


# Get the page with the 50 most followed Twitter accounts
def get_popular_accs():
  accs = []
  page = wikipedia.page(pageid='52247588')
  soup = BeautifulSoup(page.html(), 'html.parser')
  table = soup.find('table', attrs={'class':'wikitable sortable'})
  rows = table.find_all('tr')

  for row in rows:
    cols = row.find_all('td')
    if len(cols) == 0:
        continue
    # remove the @ infront of the accout name
    handle = cols[1].text.strip()[1:]
    # remove the [3], [4] etc.
    handle = re.sub("[\(\[].*?[\)\]]", "", handle)
    accs.append(handle)

  accs.pop()
  return accs

def get_billionares():
  # https://www.supermoney.com/richest-people-on-twitter/
  return ["WarrenBuffett", "BillGates", "elonmusk", "MikeBloomberg", "MichaelDell", "realDonaldTrump", "Oprah", "sherylsandberg",
          "richardbranson", "GuyKawasaki", "ThisIsSethsBlog", "Lord_Sugar", "SuzeOrmanShow", "UncleRUSH", "JerryBrownGov",
          "marcuslemonis", "LarryEllison", "marissamayer", "biz", "neiltyson", "reidhoffman", "pierre", "Carl_C_Icahn"]

def get_senates():
  # https://unusualwhales.com/i_am_the_senate/congress
  # https://www.kaggle.com/competitions/congressionaltweetcompetitionspring2022/overview
  #pass
  return congress_csv["twitter"]

def nancy_pelosy():
  # https://unusualwhales.com/i_am_the_senate_member/Nancy%20Pelosi
  # https://twitter.com/speakerpelosi
  return ["SpeakerPelosi"]

def new_channels():
  # contains a lot of information
  # obs. Every newchannel could be biased to some extend towards a political direction
  return []

def finance_news_channels():
  # obs! 
  # - contains a lot of information
  # - every newschannel could be biased to some extend towards a political direction
  # more "CNBCnow", 
  return ["business", "CNBC", "FT", "WSJ", "Reuters", "TheEconomist"]

#account_ids = get_popular_accs()
#account_ids = account_ids[41:50] #top 10
#account_ids = get_billionares()
account_ids = get_senates()
# cnn has so many twitter messages
print(account_ids)

0         RepJoeBarton
1           RepBecerra
2        SanfordBishop
3      BlumenauerMedia
4       SpeakerBoehner
            ...       
522     sendavidperdue
523      SenatorRounds
524           SenSasse
525     SenDanSullivan
526      senthomtillis
Name: twitter, Length: 527, dtype: object


### Scrape

In [ ]:
def init_scrape():
  tweets_df = pd.DataFrame(columns=['ID', 'Date', 'Text', 'Username'])

  start = date(2021, 10, 1) # we have just hf data from this point
  end = date(2022, 6, 17)
  count = 50_000 # max 50k tweets per account

  for account_id in tqdm(account_ids):
    since = "{date.year}-{date.month:02}-{date.day:02}".format(date=start)
    until = "{date.year}-{date.month:02}-{date.day:02}".format(date=end)
    mysearch = f'-filter:retweets since:{since} until:{until} from:{account_id}'

    myscraper = sntwitter.TwitterSearchScraper(mysearch).get_items()
    for i, tweet in enumerate(myscraper):
        if i > count:
            break
        input_df = {
          'ID': tweet.id,
          'Date': tweet.date,
          'Text': tweet.content,
          'Username': tweet.username,
        }
        tweets_df = tweets_df.append(input_df, ignore_index=True)

  print("\nTOTAL Tweets Gathered", len(tweets_df['ID']))
  tweets_df.to_csv("twitter_part_1.csv")
  return tweets_df
  

### Add Metadata from official Twitter api

In [ ]:
def get_twitter_metadata(tweets_df):
  # limited to 500k requests
  # need to enable the Elevated API of Twitter
  twitter_api = get_credentials()

  ### Add Metadata ###
  new_features = ['retweets', 'source', 'retweets', 'geo', 'author', 'likes', 'twitter_name', 'location', 'verified', 'followers', 'friends']
  tweets_df = tweets_df.reindex(columns = np.append(tweets_df.columns.values, new_features))

  #skip this part
  return tweets_df

  for index, row in tqdm(tweets_df.iterrows()):
      status = twitter_api.get_status(row['ID'], tweet_mode="extended")
      # Times the tweet is re-tweeted
      tweets_df.at[index, 'retweets'] = status.retweet_count
      # Which platform used to post
      tweets_df.at[index, 'source'] = status.source
      # Location where posted
      tweets_df.at[index, 'geo'] = status.geo
      # Name of the user posting
      tweets_df.at[index, 'author'] = status.user.name
      # Number of likes
      tweets_df.at[index, 'likes'] = status.favorite_count
      # Tweet posted (without URLs)
      #tweets_df.at[index, 'text'] = status.full_text
      # Screen name of the USER
      tweets_df.at[index, 'twitter_name'] = status.user.screen_name
      # Location
      tweets_df.at[index, 'location'] = status.user.location
      # Ff user is verified
      tweets_df.at[index, 'verified'] = status.user.verified
      # Number of followers
      tweets_df.at[index, 'followers'] = status.user.followers_count
      # Number of people the user follows
      tweets_df.at[index, 'friends'] = status.user.friends_count
  
  return tweets_df

In [ ]:
#tweets_df = get_twitter_metadata(tweets_df)
#tweets_df.drop(['text'], axis=1)
#tweets_df.head(3)

,ID,Date,Text,Username,retweets,source,retweets,geo,author,likes,twitter_name,location,verified,followers,friends,text
0,1537548453227769856,2022-06-16 21:31:32+00:00,We need to build democracy for the future—and ...,BarackObama,1012.0,Twitter Media Studio,1012.0,NaN,Barack Obama,6428.0,BarackObama,"Washington, DC",True,132092473.0,582743.0,We need to build democracy for the future—and ...
1,1537473010067771393,2022-06-16 16:31:45+00:00,"If we're going to protect our democracy, we ne...",BarackObama,1128.0,Twitter for iPhone,1128.0,NaN,Barack Obama,5022.0,BarackObama,"Washington, DC",True,132092473.0,582743.0,"If we're going to protect our democracy, we ne..."
2,1537102786890022916,2022-06-15 16:00:37+00:00,"For the 10th anniversary of DACA, I met with t...",BarackObama,1132.0,Twitter Media Studio,1132.0,NaN,Barack Obama,6823.0,BarackObama,"Washington, DC",True,132092473.0,582743.0,"For the 10th anniversary of DACA, I met with t..."


## Query

In [8]:
# Search for Item Ids in Tweet
if not os.path.isfile("/content/twitter.csv"):
  tweets_df = init_scrape()
  
  # Drop the NA
  tweets_df.dropna()

  # Just in case, remove duplicates
  tweets_df.drop_duplicates('Text', keep='first', inplace=True)

  print("TOTAL Tweets Gathered", len(tweets_df['ID']))
  print(tweets_df.head())

  # Add the Twitter Metadata from their official API
  tweets_df = get_twitter_metadata(tweets_df)
  
  # Store to file
  tweets_df.to_csv("/content/twitter.csv")
else:
  tweets_df = pd.read_csv("/content/twitter.csv")

In [ ]:
# multi .csv files concat together
#tweets_df_1 = pd.read_csv("/content/twitter-41-50.csv")
#tweets_df_2 = pd.read_csv("/content/twitter-billionares.csv")
#tweets_df_3 = pd.read_csv("/content/twitter-senates.csv")
#tweets_df = pd.concat([tweets_df_1, tweets_df_2, tweets_df_3], ignore_index=True)
#tweets_df.to_csv("/content/twitter.csv")

In [10]:
#tweets_df.to_csv("twitter_2.csv")
# remove the unnamed columns
tweets_df = tweets_df[tweets_df.columns.drop(list(tweets_df.filter(regex='Unnamed')))]

In [11]:
tweets_df.tail(3)
#tweets_df.drop(['text'], axis=1)

,ID,Date,Text,Username,retweets,source,retweets.1,geo,author,likes,twitter_name,location,verified,followers,friends
204905,1445063372819599372,2021-10-04 16:28:50+00:00,Disturbing to see @SenatorSinema treated this ...,SenThomTillis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204906,1445041364857729028,2021-10-04 15:01:23+00:00,Biden’s catch-and-release policies are fueling...,SenThomTillis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204907,1444411605463273479,2021-10-02 21:18:57+00:00,"Saddened to hear of the passing of my friend, ...",SenThomTillis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# NLP Sentiment

## Text Preprocessing

In [7]:
def contractions(string):
  # replace contractions
  s = re.sub(r"won’t", "will not", string)
  s = re.sub(r"would’t", "would not", string)
  s = re.sub(r"could’t", "could not", string)
  s = re.sub(r"cannot", "can not", string)
  s = re.sub(r"\’d", " would", string)
  s = re.sub(r"can\’t", "can not", string)
  s = re.sub(r"n\’t", " not", string)
  s = re.sub(r"\’re", " are", string)
  s = re.sub(r"\’s", " is", string)
  s = re.sub(r"\’ll", " will", string)
  s = re.sub(r"\’t", " not", string)
  s = re.sub(r"\’ve", " have", string)
  s = re.sub(r"\’m", " am", string)

  # replace accents
  string = string.replace('á', 'a')
  string = string.replace('à', 'a')
  string = string.replace('â', 'a')
  string = string.replace('é', 'e')
  string = string.replace('è', 'e')
  string = string.replace('ê', 'e')
  string = string.replace('ë', 'e')
  string = string.replace('î', 'i')
  string = string.replace('ï', 'i')
  string = string.replace('ö', 'o')
  string = string.replace('ô', 'o')
  string = string.replace('ò', 'o')
  string = string.replace('ó', 'o')
  string = string.replace('ù', 'u')
  string = string.replace('û', 'u')
  string = string.replace('ü', 'u')
  string = string.replace('ç', 'c')
  string = string.translate(string.maketrans('', '', letter.punctuation))
  return string

# hashtags
def deal_with_hashtags(string):
  # find hashtags
  hashtags = re.findall(r"#(\w+)", string)

  # segment hashtags
  seg_hash = []
  for i in hashtags:
    seg = seg_tw.segment(i)
    seg_hash.append(seg)

  # replace hashtag with segmented strings
  for i, v in enumerate(hashtags):
    hash = "#" + hashtags[i]
    replacement = seg_hash[i]
    string = re.sub(str(hash), str(replacement), string)
  
  return string

# ats
seg_tw = Segmenter(corpus="twitter")
def deal_with_ats(string):
  # find ats
  ats = re.findall(r"@(\w+)", string)

  # segment ats
  seg_at = []
  for i in ats:
    seg = seg_tw.segment(i)
    seg_at.append(seg)

  # replace hashtag with segmented strings
  for i, v in enumerate(ats):
    at = "#" + ats[i]
    replacement = seg_at[i]
    string = re.sub(str(at), str(replacement), string)
  
  return string

# replace emojis with words
with open(f'{DRIVE_PATH}/data/nlp/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(string):
    for emot in Emoji_Dict:
        string = string.replace(emot, " "+Emoji_Dict[emot].replace(":",""))
    return string
    
# replace emoticons with words
def convert_emoticons(string):
    for emot in EMOTICONS_EMO:
        string = string.replace(emot, " " + EMOTICONS_EMO[emot])
    return string

# Unicode
def remove_unicode(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    return text

# URLs
def replace_URL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

# Replace slang
""" Creates a dictionary with slangs and their equivalents and replaces them """
with open(f'{DRIVE_PATH}/data/nlp/slang.txt') as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2])
    for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True) # longest first for regex
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replace_slang = partial(regex.sub, lambda m: slang_map[m.group(1)])

def preprocessing(comment_text):
    # Lower case
    comment_text = ' '.join(x.lower() for x in str(comment_text).split())
    
    # Replace contractions and other replacements
    comment_text = contractions(comment_text)

    # Hashtags
    comment_text = deal_with_hashtags(comment_text)

    # Ats
    comment_text = deal_with_ats(comment_text)

    # Remove emojis
    comment_text = convert_emojis_to_word(comment_text)

    # Remove emoticons
    comment_text = convert_emoticons(comment_text)

    # Remove Unicode
    comment_text = remove_unicode(comment_text)

    # Replace URLs
    comment_text = replace_URL(comment_text)

    # Remove non-alphabet characters
    comment_text = " ".join([re.sub("[^A-Za-z]+","", x) for x in nltk.word_tokenize(comment_text)])

    # Remove extra spaces
    comment_text = re.sub(' +', " ", comment_text)

    return comment_text

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [12]:
# Apply pre-processing
tweets_df['Text'] = tweets_df['Text'].apply(lambda x: preprocessing(x))
# Apply datetime
tweets_df['Date'] = pd.to_datetime(tweets_df['Date'])

## NLP Method

In [13]:
tweets_df['Date'].min().replace(microsecond=0, second=0, minute=0) - timedelta(hours=1)

Timestamp('2021-09-30 23:00:00+0000', tz='UTC')

In [14]:
### Classic Vader Sentiment (three classes back of sentiment) ###
vader = SentimentIntensityAnalyzer()
# adding custom words
#vader.lexicon.update(new_words)

In [15]:
### GoEmotions ###
from transformers import BertTokenizer, AutoModelForSequenceClassification, pipeline

model_name = 'original' #'ekman'
tokenizer = BertTokenizer.from_pretrained(f"monologg/bert-base-cased-goemotions-{model_name}")
model = AutoModelForSequenceClassification.from_pretrained(f"monologg/bert-base-cased-goemotions-{model_name}", num_labels=28)

goemotions = pipeline(
    model=model, 
    tokenizer=tokenizer, 
    task="text-classification",
    # handing back all the scores regardless of precision
    return_all_scores=True,
    function_to_apply='sigmoid',
)

texts = [
  "Hey that's a thought! Maybe we need [NAME] to be the celebrity vaccine endorsement!",
  "it’s happened before?! love my hometown of beautiful new ken 😂😂",
  "I love you, brother.",
  "Troll, bro. They know they're saying stupid shit. The motherfucker does nothing but stink up libertarian subs talking shit",
]

res = goemotions(texts)
print("Output =", res[0])
print("Total Emotions =", len(res[0]))

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/182 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/text_classification.py:90: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  "`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality", UserWarning


Output = [{'label': 'admiration', 'score': 0.0014043459668755531}, {'label': 'amusement', 'score': 0.009338418953120708}, {'label': 'anger', 'score': 0.000517444044817239}, {'label': 'annoyance', 'score': 0.0007261092541739345}, {'label': 'approval', 'score': 0.004904965404421091}, {'label': 'caring', 'score': 0.00011518649262143299}, {'label': 'confusion', 'score': 0.029295749962329865}, {'label': 'curiosity', 'score': 0.001057165558449924}, {'label': 'desire', 'score': 0.001324195065535605}, {'label': 'disappointment', 'score': 0.0014053541235625744}, {'label': 'disapproval', 'score': 0.001127998810261488}, {'label': 'disgust', 'score': 0.00047857046592980623}, {'label': 'embarrassment', 'score': 0.0006879239808768034}, {'label': 'excitement', 'score': 0.0030392769258469343}, {'label': 'fear', 'score': 0.0005333302542567253}, {'label': 'gratitude', 'score': 9.355282236356288e-05}, {'label': 'grief', 'score': 0.0008070811745710671}, {'label': 'joy', 'score': 0.0007896315073594451}, {'

In [46]:
def is_midnight(current_time):
  seconds_since_midnight = (current_time - current_time.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
  return seconds_since_midnight == 0

# Iterate through each timeframe (days/hours/minutes)
# then Aggregate/Compound the sentiment
def get_mean_sentiment_timeframe(tweet_df, minutes):
  interval = timedelta(minutes=minutes)

  # Round down to the previous/next TimeDelta (days/hours/minutes)
  first_date = tweet_df['Date'].min().replace(microsecond=0, second=0, minute=0) - interval
  last_date = tweet_df['Date'].max().replace(microsecond=0, second=0, minute=0) + interval

  mean_sentiment_df = pd.DataFrame(columns=['Date', 'Vader-Neg', 'Vader-Neu', 'Vader-Pos', 'Vader-Compound',
    'Go-Admiration', 'Go-Amusement', 'Go-Anger', 'Go-Annoyance', 'Go-Approval',
    'Go-Caring', 'Go-Confusion', 'Go-Curiosity', 'Go-Desire', 'Go-Disappointment',
    'Go-Disapproval', 'Go-Disgust', 'Go-Embarrassment', 'Go-Excitement',
    'Go-Fear', 'Go-Gratitude', 'Go-Grief', 'Go-Joy', 'Go-Love', 'Go-Nervousness',
    'Go-Optimism', 'Go-Pride', 'Go-Realization', 'Go-Relief', 'Go-Remorse',
    'Go-Sadness', 'Go-Surprise', 'Go-Neutral'])
  while (first_date < last_date):
    # Log out if we have reached midnight for progress
    if is_midnight(first_date):
      print(f"Date {first_date}")
      # For every first date of month store it
      mean_sentiment_df.to_csv(f"/content/mean_sentiment_df_{minutes}min_{first_date}.csv")

    # Filter the twitter dataframe within this time-interval
    tweets_df_local = tweet_df[(tweet_df['Date'] >= first_date) & (tweet_df['Date'] <= first_date + interval)]

    # Iterate and Calculate Mean Sentiment accross the Text
    # Calculating the sentiment of every word in comments n combining them
    score_vader_cmnt = {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
    score_gomotions_cmnt = {'admiration': 0.0, 'amusement': 0.0, 'anger': 0.0, 'annoyance': 0.0, 'approval': 0.0,
                            'caring': 0.0, 'confusion': 0.0, 'curiosity': 0.0, 'desire': 0.0, 'disappointment': 0.0,
                            'disapproval': 0.0, 'disgust': 0.0, 'embarrassment': 0.0, 'excitement': 0.0,
                            'fear': 0.0, 'gratitude': 0.0, 'grief': 0.0, 'joy': 0.0, 'love': 0.0, 'nervousness': 0.0,
                            'optimism': 0.0, 'pride': 0.0, 'realization': 0.0, 'relief': 0.0, 'remorse': 0.0,
                            'sadness': 0.0, 'surprise': 0.0, 'neutral': 0.0}
    nr_of_tweets = 0
    for index, row in tweets_df_local.iterrows():
      ## Vader Sentiment ##
      score = vader.polarity_scores(row['Text'])
      for key, _ in score.items():
        score_vader_cmnt[key] += score[key]
      
      ## GoMotions ##
      try:
        result_go_emotions = goemotions(row['Text'])
        for row_dict in result_go_emotions:
          score_gomotions_cmnt[row_dict['label']] += row_dict['score']
      except Exception as e:
        print("ERROR, GO", str(e))
      
      # increase count
      nr_of_tweets += 1

    # calculating avg. across all the words in every comment
    try:
      for key in score_vader_cmnt:
        score_vader_cmnt[key] = score_vader_cmnt[key] / nr_of_tweets

      for key in score_gomotions_cmnt:
        score_gomotions_cmnt[key] = score_gomotions_cmnt[key] / nr_of_tweets
    except: 
      # handles: ZeroDivisionError: float division by zero
      pass

    # Add ouput to df
    input_df = {
      'Date': first_date,
      ### Vader Sentiment ###
      'Vader-Neg': score_vader_cmnt['neg'],
      'Vader-Neu': score_vader_cmnt['neu'],
      'Vader-Pos': score_vader_cmnt['pos'],
      'Vader-Compound': score_vader_cmnt['compound'],
      ### GoEmotion Sentiment ###
      'Go-Admiration': score_gomotions_cmnt['admiration'],
      'Go-Amusement': score_gomotions_cmnt['amusement'],
      'Go-Anger': score_gomotions_cmnt['anger'],
      'Go-Annoyance': score_gomotions_cmnt['annoyance'],
      'Go-Approval': score_gomotions_cmnt['approval'],
      'Go-Caring': score_gomotions_cmnt['caring'],
      'Go-Confusion': score_gomotions_cmnt['confusion'],
      'Go-Curiosity': score_gomotions_cmnt['curiosity'],
      'Go-Desire': score_gomotions_cmnt['desire'],
      'Go-Disappointment': score_gomotions_cmnt['disappointment'],
      'Go-Disapproval': score_gomotions_cmnt['disapproval'],
      'Go-Disgust': score_gomotions_cmnt['disgust'],
      'Go-Embarrassment': score_gomotions_cmnt['embarrassment'],
      'Go-Excitement': score_gomotions_cmnt['excitement'],
      'Go-Fear': score_gomotions_cmnt['fear'],
      'Go-Gratitude': score_gomotions_cmnt['gratitude'],
      'Go-Grief': score_gomotions_cmnt['grief'],
      'Go-Joy': score_gomotions_cmnt['joy'],
      'Go-Love': score_gomotions_cmnt['love'],
      'Go-Nervousness': score_gomotions_cmnt['nervousness'],
      'Go-Optimism': score_gomotions_cmnt['optimism'],
      'Go-Pride': score_gomotions_cmnt['pride'],
      'Go-Realization': score_gomotions_cmnt['realization'],
      'Go-Relief': score_gomotions_cmnt['relief'],
      'Go-Remorse': score_gomotions_cmnt['remorse'],
      'Go-Sadness': score_gomotions_cmnt['sadness'],
      'Go-Surprise': score_gomotions_cmnt['surprise'],
      'Go-Neutral': score_gomotions_cmnt['neutral'],
    }
    #print(input_df)
    mean_sentiment_df = mean_sentiment_df.append(input_df, ignore_index=True)

    # Update with interval size
    first_date = first_date + interval

  return mean_sentiment_df

In [17]:
print(len(tweets_df["Text"]))

204908


In [40]:
filtered_dataframe = tweets_df[tweets_df['Date'] >= "2022-03-01"]
filtered_dataframe = filtered_dataframe.sort_values(by='Date')

In [47]:
## Get the Mean Sentiment for a given timefractal (ie. 5min or 60min)
def get_sentiment_df(minutes):
  mean_sentiment_df = get_mean_sentiment_timeframe(filtered_dataframe, minutes)
  mean_sentiment_df.to_csv(f"/content/mean_sentiment_df_{minutes}min.csv")

In [ ]:
## Get 5min for the Log-Return ##
## Every Months at Least 60min
get_sentiment_df(5)

Date 2022-03-01 00:00:00+00:00
Date 2022-03-02 00:00:00+00:00
Date 2022-03-03 00:00:00+00:00
Date 2022-03-04 00:00:00+00:00
Date 2022-03-05 00:00:00+00:00
Date 2022-03-06 00:00:00+00:00
Date 2022-03-07 00:00:00+00:00
Date 2022-03-08 00:00:00+00:00
Date 2022-03-09 00:00:00+00:00
Date 2022-03-10 00:00:00+00:00
Date 2022-03-11 00:00:00+00:00
Date 2022-03-12 00:00:00+00:00
Date 2022-03-13 00:00:00+00:00
Date 2022-03-14 00:00:00+00:00
Date 2022-03-15 00:00:00+00:00
Date 2022-03-16 00:00:00+00:00
Date 2022-03-17 00:00:00+00:00
Date 2022-03-18 00:00:00+00:00
Date 2022-03-19 00:00:00+00:00
Date 2022-03-20 00:00:00+00:00


In [ ]:
## Get 15min for the Log-Return ##
get_sentiment_df(15)

In [ ]:
## Get 60min/1hour for the Higher Moments ##
get_sentiment_df(60)